In [1]:
import ctypes
from ctypes import wintypes as wt
from ctypes import sizeof
from time import sleep, time

In [2]:
INPUT_MOUSE = 0
INPUT_KEYBOARD = 1
INPUT_HARDWARE = 2

MOUSEEVENTF_ABSOLUTE = 0x8000
MOUSEEVENTF_LEFTDOWN = 0x0002 
MOUSEEVENTF_LEFTUP = 0x0004
MOUSEEVENTF_MOVE = 0x0001
MOUSEEVENTF_RIGHTDOWN = 0x0008
MOUSEEVENTF_RIGHTUP = 0x0010
MOUSEEVENTF_VIRTUALDESK = 0x4000

In [3]:
ULONG_PTR = ctypes.POINTER(ctypes.c_ulong)
LONG = ctypes.c_long

# These depend on whether UNICODE is defined
LPCTSTR = wt.LPCSTR  # LPCWSTR or LPCSTR
LPTSTR = wt.LPSTR  # LPWSTR or LPSTR

WNDENUMPROC = ctypes.WINFUNCTYPE(wt.BOOL, wt.HWND, wt.LPARAM)

In [4]:
class KEYBDINPUT(ctypes.Structure):
    _fields_ = [
        ('wVk', wt.WORD),
        ('wScan', wt.WORD),
        ("dwFlags", wt.DWORD),
        ("time", wt.DWORD),
        ("dwextraInfo", ULONG_PTR)
    ]
    
class MOUSEINPUT(ctypes.Structure):
    _fields_ = [
        ('dx', LONG),
        ('dy', LONG),
        ('mouseData', wt.DWORD),
        ('dwFlags', wt.DWORD),
        ('time', wt.DWORD),
        ('dwExtraInfo', ULONG_PTR),
    ]
    
class HARDWAREINPUT(ctypes.Structure):
    _fields_ = [
        ('uMsg', wt.DWORD),
        ('wParamL', wt.WORD),
        ('wParamH', wt.WORD)
    ]
    

In [5]:
class InptUnion(ctypes.Union):
    _fields_ = [
        ('mi', MOUSEINPUT),
        ('ki', KEYBDINPUT),
        ('hi', HARDWAREINPUT)
    ]

class INPUT(ctypes.Structure):
    _fields_ = [
        ('type', wt.DWORD),
        ('ip', InptUnion)
    ]
    
PINPUT = ctypes.POINTER(INPUT)

In [6]:
def make_kb_input(keycode, scan_code=0, flags=0):
    ip = INPUT()
    ip.type = INPUT_KEYBOARD
    
    ki = KEYBDINPUT()
    ki.wVk = keycode
    ki.wScan = scan_code
    ki.dwFlags = flags
    
    ip.ip.ki = ki
    return ip    

In [7]:
# Raw exported windows API functions

user32 = ctypes.windll.user32
k32 = ctypes.windll.kernel32

_SendInput = user32.SendInput
_SendInput.argtypes = [wt.UINT, PINPUT, ctypes.c_int]
_SendInput.restype = wt.UINT

def SendInput(*args):
    res = _SendInput(*args)
    if not res:
        raise OSError(k32.GetLastError())
    return res

FindWindow = user32.FindWindowA
FindWindow.argtypes = [LPCTSTR, LPCTSTR]
FindWindow.restype = wt.HWND

CloseHandle = k32.CloseHandle
CloseHandle.argtypes = [wt.HANDLE]
CloseHandle.restype = wt.BOOL

EnumWindows = user32.EnumWindows
EnumWindows.argtypes = [WNDENUMPROC, wt.LPARAM]
EnumWindows.restype = wt.BOOL

GetWindowText = user32.GetWindowTextA
GetWindowText.argtypes = [wt.HWND, LPTSTR, ctypes.c_int]
GetWindowText.restype = ctypes.c_int

SetActiveWindow = user32.SetActiveWindow
SetActiveWindow.argtypes = [wt.HWND]
SetActiveWindow.restype = wt.HWND

SetForegroundWindow = user32.SetForegroundWindow
SetForegroundWindow.argtypes = [wt.HWND]
SetForegroundWindow.restype = wt.BOOL

GetLastError = k32.GetLastError
GetLastError.argtypes = []
GetLastError.restype = ctypes.c_int

In [8]:
def make_mouse_move(x,y):
    ip = INPUT()
    ip.type = INPUT_MOUSE
    
    mi = MOUSEINPUT()
    mi.dx = x
    mi.dy = y
    mi.mouseData = 0
    mi.time = 0
    mi.dwFlags=(MOUSEEVENTF_MOVE) | MOUSEEVENTF_ABSOLUTE | MOUSEEVENTF_VIRTUALDESK
    ip.ip.mi = mi
    return ip

In [21]:
def make_mouse_left_click(dwFlags):
    ip = INPUT()
    ip.type = INPUT_MOUSE
    
    mi = MOUSEINPUT()
    mi.dx = 0
    mi.dy = 0
    mi.mouseData = 0
    mi.time = 0
    mi.dwFlags = MOUSEEVENTF_ABSOLUTE | MOUSEEVENTF_VIRTUALDESK | dwFlags
    ip.ip.mi = mi
    return ip

In [24]:
import win32api
SCREEN_X = win32api.GetSystemMetrics(78)
SCREEN_Y = win32api.GetSystemMetrics(79)

def norm(p):
    return int(p.x / (SCREEN_X) * 65535), int(p.y / (SCREEN_Y) * 65536)

def norm2(p):
    return int(p[0] / (SCREEN_X) * 65535), int(p[1] / (SCREEN_Y) * 65536)

In [25]:
def send_keyboard_input(keycode, scan_code=0, flags=0):
    ip = make_kb_input(keycode, scan_code, flags)
    return SendInput(1, PINPUT(ip), ctypes.sizeof(ip))

def mouse_move(x,y):
    ip = make_mouse_move(x,y)
    return SendInput(1, PINPUT(ip), ctypes.sizeof(ip))
    
def mouse_click():
    ip2 = make_mouse_left_click(MOUSEEVENTF_LEFTDOWN)
    ip3 = make_mouse_left_click(MOUSEEVENTF_LEFTUP)
    send_inputs(ip2, ip3)

def send_inputs(*ips):
    sz = ctypes.sizeof(ips[0])
    array = (INPUT * len(ips))(*ips)
    return SendInput(len(ips), array, sz)

In [26]:
def get_pos():
    point = ctypes.wintypes.POINT()
    ctypes.windll.user32.GetCursorPos(ctypes.byref(point))
    x, y = norm(point)
    return x,y

def get_pos2():
    point = ctypes.wintypes.POINT()
    ctypes.windll.user32.GetCursorPos(ctypes.byref(point))
    return point.x, point.y

In [27]:
def run_clicks(wait=0.05):
    xs, ys = get_pos()
    try:
        while True:
            mouse_click()
            sleep(wait)
        #sleep(5)
            xs2, xy2 = get_pos()
            if xs2 != xs or xy2 != ys:
                return
    except KeyboardInterrupt:
        pass

def run_clicks2(x, y, wait=0.05):
    while True:
        x1, y1 = get_pos()
        mouse_move(x, y)
        mouse_click()
        mouse_move(x1, y1)
        sleep(wait)
        if win32api.GetKeyState(0x02) & 0x8000:
            return
        

In [28]:
def clickfind():
    down = False
    while True:
        try:
            sleep(0.01)
            print("\r    (%d, %d),"%get_pos(), end="")
            if win32api.GetKeyState(0xC0) & 0x8000:
                if not down:
                    print()
                    down = True
            else:
                down = False
        except:
            break

In [41]:
buttons = [
    (29780, 22300),
    (29652, 25637),
    (29823, 29430),
    (29908, 33147),
    (29908, 36864),
    (24106, 40884),
]

for x,y in buttons:
    mouse_move(x,y)
    sleep(.3)

In [49]:
def right_click_down():
    return win32api.GetKeyState(0x02) & 0x8000

buttons = [
    (29780, 22300),
    (29652, 25637),
    (29823, 29430),
    (29908, 33147),
    (29908, 36864),
    (24106, 40884),
]

CLICK_COORD = buttons[5]

SLEEP_TIME = 0.05
TAX = 0
CALL_TO_ARMS = 1
HOLY_LIGHT = 2
GODS_HAND = 3
DIAMOND_AXE = 4

def wait(s):
    end = time() + s
    while True:
        left = end - time()
        if left > 0:
            sleep(min(left, 0.1))
        else:
            break
            
def autoclick(idx):
    x, y = buttons[idx]
    mouse_move(x,y)
    mouse_click()
            
class StopClick(Exception):
    pass

def click_for(s):
    x, y = CLICK_COORD
    mouse_move(x,y)
    end = time() + s
    ST = SLEEP_TIME
    while True:
        if right_click_down():
            raise StopClick()
        else:
            mouse_click()
        if time() > end:
            break
        sleep(ST)
        
def click_angels_66mps(mps=66, mana=1900):
    s1 = 300+800
    s2 = 800+900
    safety = 0.1
    t1 = (s2 - (mana - s1)) / mps + safety
    t2 = mana / mps + safety
    while True:
        try:
            autoclick(GODS_HAND)
            autoclick(CALL_TO_ARMS)
            click_for(t1)
            autoclick(HOLY_LIGHT)
            autoclick(DIAMOND_AXE)
            click_for(t2)
        except StopClick:
            break  

In [50]:
click_angels_66mps(69.54, 1919)